In [ ]:
#       PHONEPE PULSE DATA DOWNLOAD FROM GITHUB REPOSITORY  -  Executed once to download data into local workspace
import git

repository_url = "https://github.com/PhonePe/pulse.git"
destination_directory = "C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github"
#destination_directory = "D:\Krithiga\Proj2_PhonePe\Data_github"
git.Repo.clone_from(repository_url, destination_directory)
#### data extracted

In [1]:
###    Map Indian state names from phonepe pulse data to Indian state names in geojson file(for visualisation)    

state_data_map={'andaman-&-nicobar-islands'         :'Andaman & Nicobar',
                'andhra-pradesh'                    :'Andhra Pradesh',
                'arunachal-pradesh'                 :'Arunachal Pradesh',
                'assam'                             :'Assam',
                'bihar'                             :'Bihar',
                'chandigarh'                        :'Chandigarh',
                'chhattisgarh'                      :'Chhattisgarh',
                'dadra-&-nagar-haveli-&-daman-&-diu':'Dadra and Nagar Haveli and Daman and Diu',
                'delhi'                             :'Delhi',
                'goa'                               :'Goa',
                'gujarat'                           :'Gujarat',
                'haryana'                           :'Haryana',
                'himachal-pradesh'                  :'Himachal Pradesh',
                'jammu-&-kashmir'                   :'Jammu & Kashmir',
                'jharkhand'                         :'Jharkhand',
                'karnataka'                         :'Karnataka',
                'kerala'                            :'Kerala',
                'ladakh'                            :'Ladakh',
                'lakshadweep'                       :'Lakshadweep',
                'madhya-pradesh'                    :'Madhya Pradesh',
                'maharashtra'                       :'Maharashtra',
                'manipur'                           :'Manipur',
                'meghalaya'                         :'Meghalaya',
                'mizoram'                           :'Mizoram',
                'nagaland'                          :'Nagaland',
                'odisha'                            :'Odisha',
                'puducherry'                        :'Puducherry',
                'punjab'                            :'Punjab',
                'rajasthan'                         :'Rajasthan',
                'sikkim'                            :'Sikkim',
                'tamil-nadu'                        :'Tamil Nadu',
                'telangana'                         :'Telangana',
                'tripura'                           :'Tripura',
                'uttar-pradesh'                     :'Uttarakhand',
                'uttarakhand'                       :'Uttar Pradesh',
                'west-bengal'                       :'West Bengal'
                }

In [6]:
##      Import necessary libraries for accessing/reading/manipulating data

import pandas as pd
import json
import os

In [4]:
##  Establish MySQL Connection Thread
import mysql.connector

mydb = mysql.connector.connect( host="localhost", user="root", password="",database="project_phonepe")
mycursor = mydb.cursor(buffered=True)

In [7]:
##      Create a separate database for this project data tables

#mycursor.execute('create database project_phonepe')

In [ ]:
#       Extract Aggregate Transaction Data - for each state - Year/Quarter wise

clm={'State':[], 'Year':[],'Quarter':[],'Transaction_Type':[], 'Transaction_Count':[], 'Transaction_Amount':[]}

path_agg_state="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/aggregated/transaction/country/india/state/"   
Agg_state_list=os.listdir(path_agg_state)
for i in Agg_state_list:
    p_i=path_agg_state+i+"/"
    Agg_yr_list=os.listdir(p_i)
    for j in Agg_yr_list:
        p_j=p_i+j+"/"
        Agg_yr_qtr_list=os.listdir(p_j)
        for k in Agg_yr_qtr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['transactionData']:
              Name=z['name']
              count=z['paymentInstruments'][0]['count']
              amount=z['paymentInstruments'][0]['amount']
              clm['Transaction_Type'].append(Name)
              clm['Transaction_Count'].append(count)
              clm['Transaction_Amount'].append(amount)
              clm['State'].append(state_data_map[i])
              clm['Year'].append(j)
              clm['Quarter'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_Trans_df=pd.DataFrame(clm)

In [ ]:
##  Create Table for Aggregate Transaction Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Aggregate_Transaction (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, Transaction_Type VARCHAR(255), Transaction_Count INT, 
                 Transaction_Amount BIGINT)''')

In [ ]:
##  Insert the Aggregate Transactions data extracted from json into SQL database

for index,row in Agg_Trans_df.iterrows():
    mycursor.execute('''INSERT INTO Aggregate_Transaction(State, Year,Quarter, Transaction_Type, Transaction_Count, 
                     Transaction_Amount) VALUES (%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['Transaction_Type'],
                      row['Transaction_Count'],row['Transaction_Amount']))
    
mydb.commit()

In [ ]:
#       Extract Aggregate User Data for each State - Year/Quarter wise (including brandwise data)

clm1={'State':[], 'Year':[],'Quarter':[],'Registered_Users':[], 'App_Opens':[]}
clm2={'State':[], 'Year':[],'Quarter':[],'Brand_Name':[], 'Brand_User_Count':[], 'Registered_Users':[], 'Brand_User_Percentage':[]}

path_user_state="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/aggregated/user/country/india/state/"   
Agg_state_list=os.listdir(path_user_state)
for i in Agg_state_list:
    p_i=path_user_state+i+"/"
    Agg_yr_list=os.listdir(p_i)
    for j in Agg_yr_list:
        p_j=p_i+j+"/"
        Agg_yr_qtr_list=os.listdir(p_j)
        for k in Agg_yr_qtr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            reg_users=D['data']['aggregated']['registeredUsers']
            clm1['State'].append(state_data_map[i])
            clm1['Year'].append(j)
            clm1['Quarter'].append(int(k.strip('.json')))
            clm1['Registered_Users'].append(reg_users)
            clm1['App_Opens'].append(D['data']['aggregated']['appOpens'])
            if (D['data']['usersByDevice']) is not None:
                for z in D['data']['usersByDevice']:
                    clm2['Brand_Name'].append(z['brand'])
                    clm2['Brand_User_Count'].append(z['count'])
                    clm2['Registered_Users'].append(reg_users)
                    clm2['Brand_User_Percentage'].append(z['percentage'])
                    clm2['State'].append(state_data_map[i])
                    clm2['Year'].append(j)
                    clm2['Quarter'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_User_df=pd.DataFrame(clm1)
Agg_Brand_User_df=pd.DataFrame(clm2)

In [ ]:
##  Create Table for Aggregate User Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Aggregate_Users (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, Registered_Users INT, App_Opens BIGINT)''')

In [ ]:
##      Insert Aggregate User data extracted into SQL Database

for index,row in Agg_User_df.iterrows():
    mycursor.execute('''INSERT INTO Aggregate_Users(State, Year, Quarter, Registered_Users, App_Opens) 
                     VALUES (%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['Registered_Users'],row['App_Opens']))
    
mydb.commit()

In [ ]:
##  Create Table for Aggregate Brandwise User Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Aggregate_Brand_Users (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, Brand_Name VARCHAR(255), Brand_User_Count INT, 
                 Registered_Users INT, Brand_User_Percentage FLOAT)''')

In [ ]:
##      Insert Aggregate Brandwise User data extracted into SQL Database

for index,row in Agg_Brand_User_df.iterrows():
    mycursor.execute('''INSERT INTO Aggregate_Brand_Users(State, Year, Quarter, Brand_Name, Brand_User_Count, 
                     Registered_Users, Brand_User_Percentage) VALUES (%s,%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['Brand_Name'],row['Brand_User_Count'],
                      row['Registered_Users'],row['Brand_User_Percentage']))
    
mydb.commit()

In [ ]:
#       Extract Transaction Data for all Districts of each States - Year/Quarter Wise


clm3={'State':[], 'Year':[],'Quarter':[],'District':[], 'Transaction_Count':[], 'Transaction_Amount':[]}

path_map_txn="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/map/transaction/hover/country/india/state/"   
Map_state_list=os.listdir(path_map_txn)
for i in Map_state_list:
    p_i=path_map_txn+i+"/"
    Map_yr=os.listdir(p_i)
    for j in Map_yr:
        p_j=p_i+j+"/"
        Map_yr_list=os.listdir(p_j)
        for k in Map_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['hoverDataList']:
                clm3['State'].append(state_data_map[i])
                clm3['Year'].append(j)
                clm3['Quarter'].append(int(k.strip('.json')))
                clm3['District'].append(z['name'])
                clm3['Transaction_Count'].append(z['metric'][0]['count'])
                clm3['Transaction_Amount'].append(z['metric'][0]['amount'])
                                    
#Succesfully created a dataframe
Map_Trans_df=pd.DataFrame(clm3)

In [ ]:
##  Create Table for Map Transaction Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Map_Transaction (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, District VARCHAR(255), Transaction_Count INT, 
                 Transaction_Amount BIGINT)''')

In [ ]:
##      Insert Map State/District wise Transaction data extracted into SQL Database

for index,row in Map_Trans_df.iterrows():
    mycursor.execute('''INSERT INTO Map_Transaction(State, Year,Quarter, District, Transaction_Count, Transaction_Amount) 
                     VALUES (%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['District'],row['Transaction_Count'],row['Transaction_Amount']))
    
mydb.commit()

In [ ]:
#       Extract Map User Data for each State/District - Year/Quarter wise (including brandwise data)

clm4={'State':[], 'Year':[], 'Quarter':[], 'District':[], 'Registered_Users':[], 'App_Opens':[]}


path_map_user="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/map/user/hover/country/india/state/"   
Map_state_list=os.listdir(path_map_user)
for i in Map_state_list:
    p_i=path_map_user+i+"/"
    Map_yr_list=os.listdir(p_i)
    for j in Map_yr_list:
        p_j=p_i+j+"/"
        Map_yr_qtr_list=os.listdir(p_j)
        for k in Map_yr_qtr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            key_dist_list=list(D['data']['hoverData'].keys())
            for district in key_dist_list:
                clm4['State'].append(state_data_map[i])
                clm4['Year'].append(j)
                clm4['Quarter'].append(int(k.strip('.json')))
                clm4['District'].append(district)
                clm4['Registered_Users'].append(D['data']['hoverData'][district]['registeredUsers'])
                clm4['App_Opens'].append(D['data']['hoverData'][district]['appOpens'])
#Succesfully created a dataframe
Map_User_df=pd.DataFrame(clm4)

In [ ]:
##  Create Table for Map User Data(State-District)

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Map_Users (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, District VARCHAR(255), Registered_Users INT, App_Opens BIGINT)''')

In [ ]:
##      Insert Map User data extracted into SQL Database(State-District)

for index,row in Map_User_df.iterrows():
    mycursor.execute('''INSERT INTO Map_Users(State, Year, Quarter, District, Registered_Users, App_Opens) 
                     VALUES (%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['District'],row['Registered_Users'],row['App_Opens']))
    
mydb.commit()

In [ ]:
#       Extract Transaction Data for top 10 Districts/Pincodes of each States - Year/Quarter Wise


clm5={'State':[], 'Year':[],'Quarter':[],'District':[], 'Transaction_Count':[], 'Transaction_Amount':[]}
clm6={'State':[], 'Year':[],'Quarter':[],'Pincode':[], 'Transaction_Count':[], 'Transaction_Amount':[]}

path_top_txn="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/top/transaction/country/india/state/"   
Top_state_list=os.listdir(path_top_txn)
for i in Top_state_list:
    p_i=path_top_txn+i+"/"
    Top_yr=os.listdir(p_i)
    for j in Top_yr:
        p_j=p_i+j+"/"
        Top_yr_list=os.listdir(p_j)
        for k in Top_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)

            for z1 in D['data']['districts']:
                clm5['State'].append(state_data_map[i])
                clm5['Year'].append(j)
                clm5['Quarter'].append(int(k.strip('.json')))
                clm5['District'].append(z1['entityName'])
                clm5['Transaction_Count'].append(z1['metric']['count'])
                clm5['Transaction_Amount'].append(z1['metric']['amount'])

            for z2 in D['data']['pincodes']:
                clm6['State'].append(state_data_map[i])
                clm6['Year'].append(j)
                clm6['Quarter'].append(int(k.strip('.json')))
                clm6['Pincode'].append(z2['entityName'])
                clm6['Transaction_Count'].append(z2['metric']['count'])
                clm6['Transaction_Amount'].append(z2['metric']['amount'])
                                    
#Succesfully created a dataframe
Top_Trans_Dist_df=pd.DataFrame(clm5)
Top_Trans_Pncd_df=pd.DataFrame(clm6)

In [ ]:
##  Create Table for Top 10 Transactions of a State - District Wise Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Top_Transaction_District (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, District VARCHAR(255), Transaction_Count INT, 
                 Transaction_Amount BIGINT)''')

In [ ]:
##  Insert Values into Top 10 Transactions of a State - District Wise Database

for index,row in Top_Trans_Dist_df.iterrows():
    mycursor.execute('''INSERT INTO Top_Transaction_District(State, Year,Quarter, District, Transaction_Count, Transaction_Amount) 
                     VALUES (%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['District'],row['Transaction_Count'],row['Transaction_Amount']))
    
mydb.commit()

In [ ]:
##  Create Table for Top 10 Transactions of a State - District Wise Data

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Top_Transaction_Pincode (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, Pincode INT, Transaction_Count INT, Transaction_Amount BIGINT)''')

In [ ]:
##  Insert Values into Top 10 Transactions of a State - District Wise Database

for index,row in Top_Trans_Pncd_df.iterrows():
    mycursor.execute('''INSERT INTO Top_Transaction_Pincode(State,Year,Quarter,Pincode,Transaction_Count,Transaction_Amount) 
                     VALUES (%s,%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['Pincode'],row['Transaction_Count'],row['Transaction_Amount']))
    
mydb.commit()

In [ ]:
#       Extract Top User Data for each State - District/Pincode - Year/Quarter wise

clm7={'State':[], 'Year':[], 'Quarter':[], 'District':[], 'Registered_Users':[]}
clm8={'State':[], 'Year':[], 'Quarter':[], 'Pincode':[], 'Registered_Users':[]}

path_top_user="C:/Users/spand/OneDrive/Desktop/Proj2_PhonePe/Data_github/data/top/user/country/india/state/"   
Top_state_list=os.listdir(path_top_user)
for i in Top_state_list:
    p_i=path_top_user+i+"/"
    Top_yr_list=os.listdir(p_i)
    for j in Top_yr_list:
        p_j=p_i+j+"/"
        Top_yr_qtr_list=os.listdir(p_j)
        for k in Top_yr_qtr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z1 in D['data']['districts']:
                clm7['State'].append(state_data_map[i])
                clm7['Year'].append(j)
                clm7['Quarter'].append(int(k.strip('.json')))
                clm7['District'].append(z1['name'])
                clm7['Registered_Users'].append(z1['registeredUsers'])

            for z2 in D['data']['pincodes']:
                clm8['State'].append(state_data_map[i])
                clm8['Year'].append(j)
                clm8['Quarter'].append(int(k.strip('.json')))
                clm8['Pincode'].append(z2['name'])
                clm8['Registered_Users'].append(z2['registeredUsers'])
                
#Succesfully created a dataframe
Top_User_District_df=pd.DataFrame(clm7)
Top_User_Pincode_df=pd.DataFrame(clm8)

In [ ]:
##  Create Table for Top Registered User Data(State-District)

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Top_Users_District (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, District VARCHAR(255), Registered_Users INT)''')

In [ ]:
##      Insert Top User data State - District wise extracted into SQL Database(State-District)

for index,row in Top_User_District_df.iterrows():
    mycursor.execute('''INSERT INTO Top_Users_District(State, Year, Quarter, District, Registered_Users) 
                     VALUES (%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['District'],row['Registered_Users']))
    
mydb.commit()

In [ ]:
##  Create Table for Top Registered User Data(State-Pincode)

mycursor.execute('''create table IF NOT EXISTS project_phonepe.Top_Users_Pincode (State VARCHAR(255),
                 Year SMALLINT, Quarter TINYINT, Pincode INT, Registered_Users INT)''')

In [ ]:
##      Insert Top User data State - Pincode wise extracted into SQL Database(State-District)

for index,row in Top_User_Pincode_df.iterrows():
    mycursor.execute('''INSERT INTO Top_Users_Pincode(State, Year, Quarter, Pincode, Registered_Users) 
                     VALUES (%s,%s,%s,%s,%s)''',
                     (row['State'],row['Year'],row['Quarter'],row['Pincode'],row['Registered_Users']))
    
mydb.commit()